### Scopo del notebook:
In questo notebook verrà aumentato il numero di porzioni di foto degli elementi scarti. Anziché dividere la foto in n porzioni, verrà considerata una finestra m*m e fatta scorrere. Considerando anche le maschere create per decidere se può essere una porzione scarta o meno.

In [1]:
import cv2
import numpy as np
import glob
import math
from matplotlib import pyplot as plt

In [2]:
def cutWorkSpace(img,border,mask): 
    # dimensioni immagini
    img_h = 2048
    img_l = 2048
    # offset per posizionamento su area d'indagine
    offset_x = 600
    offset_y = 500
    
    #taglio per garantire la ricerca sull'area di interesse, evitando rumori dati dallo sfondo
    crop_img = img[offset_y:img_h-offset_y, offset_x:img_l-offset_x]
    
    # Convert the img to grayscale
    gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    high_thresh, thresh_im = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #cv2.imwrite('provaT.jpg', thresh_im)
    lowThresh = 0.5*high_thresh
    
    
    # Apply edge detection method on the image
    #edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    cv2.imwrite('canny.jpg', edges)
    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges,1,np.pi/180, 100)
    #cv2.imwrite('Risultati/resFS'+name+'c.jpg', edges)
    # The below for loop runs till r and theta values 
    # are in the range of the 2d array
    i = 0
    x_first = 0
    x_second = 0
    count = 0
    delta = 30
    thetaFinal = 0
    while count < 2:
        for r,theta in lines[i]:
            # Stores the value of cos(theta) in a
            a = np.cos(theta)

            # Stores the value of sin(theta) in b
            b = np.sin(theta)

            # x0 stores the value rcos(theta)
            x0 = a*r

            # y0 stores the value rsin(theta)
            y0 = b*r

            #ti define line lenght
            const = 3000
            # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
            x1 = int(x0 + const*(-b))

            # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
            y1 = int(y0 + const*(a))

            # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
            x2 = int(x0 - const*(-b))

            # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
            y2 = int(y0 - const*(a))

            if count == 0 :
                if theta != 0:
                    thetaFinal = 180-math.degrees(theta)
                x_first = x0
                #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                count += 1
            else:
                if abs(x_first-x0) > delta :
                    x_second = x0
                    #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                    count += 1
            i += 1
    # All the changes made in the input image are finally
    # written on a new image houghlines.jpg
    #cv2.imwrite('Risultati/res'+name+'.jpg', img)
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # rotate our image by Theta degrees around the center of the image
    M = cv2.getRotationMatrix2D((cX, cY), -thetaFinal, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    rotated_mask = cv2.warpAffine(mask, M, (w, h))
    
    scale_percent = 100 # percent of original size

    
    if x_first < x_second:
        median = (x_second+x_first)//2
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        temp_mask = rotated_mask[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)  
        temp_mask = cv2.resize(temp_mask, dim, interpolation = cv2.INTER_AREA)  
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        gray_mask = cv2.cvtColor(temp_mask,cv2.COLOR_BGR2GRAY)
        return  gray, gray_mask
    else:
        median = (x_first+x_second)//2.
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        temp_mask = rotated_mask[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]        
        #blurred = cv2.GaussianBlur(temp, (3,3), 0)
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)
        temp_mask = cv2.resize(temp_mask, dim, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        gray_mask = cv2.cvtColor(temp_mask,cv2.COLOR_BGR2GRAY)
        return  gray, gray_mask

In [3]:
def divideImage(img,pcs):
    results = []
    lung = len(img) // pcs
    for i in range(pcs):
        results.append(img[(lung*i):(lung*(i+1)-1)][:])

    return results

In [4]:
names = {#2: ['Cam3@00004 2.tif','Cam3@00004 2_M.tif'],
         3: ['Cam3@00004 3.tif','Cam3@00004 3_M.tif'],
         4: ['Cam3@00004 4.tif','Cam3@00004 4_M.tif'],
         5: ['Cam3@00004 5.tif','Cam3@00004 5_M.tif'],
         6: ['Cam3@00004 6.tif','Cam3@00004 6_M.tif'],
         7: ['Cam3@00004 7.tif','Cam3@00004 7_M.tif'],
         8: ['Cam3@00004 8.tif','Cam3@00004 8_M.tif'],
         9: ['Cam3@00004 9.tif','Cam3@00004 9_M.tif']
         #10: ['Cam3@00004 10.tif','Cam3@00004 10_M.tif']
        }

In [5]:
path = '/Users/leonardoaggio/Desktop/Dataset/Dataset/Maschere&Scarti/'

border = 60

results = []



In [6]:
images = []
masks = []

for i in names:
    image_path = path + names[i][0] 
    mask_path = path + names[i][1] 
    img = cv2.imread(image_path)
    mask = cv2.imread(mask_path)
    img,mask = cutWorkSpace(img,border,mask)
    # I pixel sono impostati a 0 o a 1 in base a un valore di default : 150
    for row in range(len(mask)):
        for col in range(len(mask[0])):
            if mask[row,col] > 150:
                mask[row,col] = 255
            else:
                mask[row,col] = 0
    images.append(img)
    masks.append(mask)
                
    #plt.imshow(img)
    #plt.show()
    #plt.imshow(mask)
    #plt.show()

Creo una finestra che viene spostata verso il basso partendo dall'inizio dell'immagine e mi crea una immagine solamente se l'intersezione tra la finestra e la maschera ha un numero di elementi = 255 maggiori di un determinato threshold

In [7]:
threshold = 2500

window_size_x = 120
window_size_y = 120

saveFile = '/Users/leonardoaggio/Desktop/Dataset/Dataset/SectionScarti/'

step = 100

In [8]:
for i in range(len(names)):
    
    start = 0
    j = 0
    #trovo la finestra sulla maschera e controllo numero di pixels neri
    while (start + window_size_y) < len(masks[i]):
    
        window = masks[i][start:start+120]
        
        count = sum( [np.count_nonzero(listElem == 0) for listElem in window])

        if count > threshold:
            temp_cut = images[i][start:start+120]
            cv2.imwrite(saveFile+str(i)+'_'+ str(j)+'.tif',temp_cut)
            j += 1
            
        start += step
        
print('DONE!')
        

DONE!
